# Laphet with myPOS Tags

Language model ကို NLP task အမျိုးမျိုးအတွက် အသုံးပြုကြပါတယ်။ ဒီ notebook မှာတော့ Laphet LM Toolkit ကို သုံးပြီး မြန်မာစာကြောင်းတွေရဲ့ Part-of-Speech prediction/generation ကို လက်တွေ့ လုပ်ပြပါမယ်။  

## Dataset Information

Link: [https://github.com/ye-kyaw-thu/myPOS](https://github.com/ye-kyaw-thu/myPOS)  

Preprocessing အနေနဲ့ myPOS dataset ထဲက tag တွေကိုပဲ ဆွဲထုတ်ယူထားပါတယ်။  
tag တွေ သို့မဟုတ် word တွေကို ဆွဲထုတ်ဖို့အတွက် သုံးခဲ့တဲ့ perl code က ဒီလင့်ကနေ ရယူနိုင်ပါတယ်။  

[https://github.com/ye-kyaw-thu/myPOS/blob/master/corpus-draft-ver-1.0/mk-wordtag.pl](https://github.com/ye-kyaw-thu/myPOS/blob/master/corpus-draft-ver-1.0/mk-wordtag.pl)  

Laphet repository မှာလည်း ကော်ပီကူးထည့်ပေးထားပါမယ်။  

In [1]:
!ls -lh ./data/myPOS/tag/*

-rw-rw-r-- 1 ye ye 103K Jan 26 18:03 ./data/myPOS/tag/dev_tag.txt
-rw-rw-r-- 1 ye ye   95 Jan 24 18:42 ./data/myPOS/tag/start_tags.txt
-rw-rw-r-- 1 ye ye  48K Jan 24 17:43 ./data/myPOS/tag/test_tag.txt
-rw-rw-r-- 1 ye ye 1.9M Jan 26 18:02 ./data/myPOS/tag/train_tag.txt


In [2]:
!wc ./data/myPOS/tag/train_tag.txt

  40000  522381 1908736 ./data/myPOS/tag/train_tag.txt


In [3]:
!wc ./data/myPOS/tag/dev_tag.txt

  2196  28668 104680 ./data/myPOS/tag/dev_tag.txt


In [4]:
!wc ./data/myPOS/tag/test_tag.txt

 1000 13468 48966 ./data/myPOS/tag/test_tag.txt


In [5]:
!wc ./data/myPOS/tag/start_tags.txt

15 28 95 ./data/myPOS/tag/start_tags.txt


## Data Format

In [6]:
!head ./data/myPOS/tag/train_tag.txt

num n v n ppm n num part v ppm punc
n part ppm v v part v part part n v n conj punc n conj n n ppm v part part n n part v part n conj n n v ppm punc
adj n ppm v part n part v ppm punc
n v part ppm v v part part n n part ppm v n part ppm n ppm adv v part v part part part punc
pron ppm pron n part punc
num n n n n punc num part punc n n punc n n punc
pron n tn part v part part ppm punc
pron part ppm pron n ppm n ppm v part v part ppm punc
n v part adj n ppm pron part part punc
adj n ppm n punc n punc n part n part v ppm punc


In [7]:
!tail ./data/myPOS/tag/dev_tag.txt

pron tn part part v ppm punc
n n n n part ppm adv v part part ppm punc
conj n part v conj adv v part part punc
pron pron v part n v part n v part part punc
n v part ppm n v ppm punc
v part ppm punc
v v part v part punc
pron ppm n n ppm v part part part part punc
pron pron part ppm n tn part part ppm n v part v part punc
pron part ppm adj n part punc


In [8]:
!head ./data/myPOS/tag/test_tag.txt

tn n ppm n tn part punc
n ppm pron pron ppm v part ppm punc
pron n v v part punc
n v part v ppm part punc
n adv v part punc tn tn n part v part conj v part ppm part punc
v part conj n ppm v part v ppm punc
n ppm v ppm punc
n v conj n v part part part punc
pron v part ppm tn part part v part ppm punc
n part ppm v conj n part ppm n n ppm n v ppm punc


In [9]:
!cat ./data/myPOS/tag/start_tags.txt

pron
n
adj
v
pron part
pron ppm
n v
n n
v part
n part
pron pron
pron ppm
n tn
adj v part
n n n


start_tags.txt ဖိုင်မှာ start word (တကယ်ကတော့ ဒီနေရာမှာ start tag ပါ) အနေနဲ့ ပေးတဲ့အခါမှာ တစ်လုံးထက်မက ပိုပေးထားတာကို ဂရုပြုပါ။  

## Bash Shell Script

Experiment အကုန်၊ ဆိုလိုတာက MLP, Bi-LSTM, Transformer, BERT, GPT မော်ဒယ်အကုန်နဲ့ experiment လုပ်မယ် ဆိုရင်တော့ အောက်ပါ shel script နဲ့ run ပါ။  

In [10]:
!cat ./train_test_tag.sh

#!/bin/bash

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001;

    echo "Text generation:";
    time python -u laphet.py --model_type $model_type --generate --model $model_file \
      --seq_len 50 --prompt "ရဲ" --no_of_generation

ဒီ notebook မှာ အကုန်လုံးကို တန်းစီပြီး တခါတည်း run ချလိုက်ရင် လိုက်ကြည့်ရခက်နေမှာစိုးလို့ မော်ဒယ် တစ်ခုချင်းစီကို သပ်သပ်စီ run ပြသွားပါမယ်။ အဲဒီလို လုပ်ဖို့အတွက် ဆိုရင် task line တွေကို comment အပိတ်အဖွင့်လုပ်ပြီး ကစားပေးရပါလိမ့်မယ်။  

## Bash Script for MLP  

In [11]:
!cat ./train_test_tag.sh

#!/bin/bash

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001;

    echo "Text generation:";
    time python -u laphet.py --model_type $model_type --generate --model $model_file \
      --seq_len 50 --prompt "ရဲ" --no_of_generation

## MLP based LM Training, Text Generation and Testing

In [12]:
!./train_test_tag.sh

Training Mlp language model:
Epoch 1/10 (Training): 100%|███████████████| 1250/1250 [00:09<00:00, 133.75it/s]
Epoch 1, Training Loss: 0.6058
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 380.24it/s]
Epoch 1, Validation Loss: 0.5729
Best model saved at ./model/tag/mlp.model with validation loss: 0.5729
Epoch 2/10 (Training): 100%|███████████████| 1250/1250 [00:08<00:00, 141.09it/s]
Epoch 2, Training Loss: 0.5729
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 531.65it/s]
Epoch 2, Validation Loss: 0.5729
Best model saved at ./model/tag/mlp.model with validation loss: 0.5729
Epoch 3/10 (Training): 100%|███████████████| 1250/1250 [00:08<00:00, 140.36it/s]
Epoch 3, Training Loss: 0.5729
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 389.54it/s]
Epoch 3, Validation Loss: 0.5729
Best model saved at ./model/tag/mlp.model with validation loss: 0.5729
Epoch 4/10 (Training): 100%|███████████████| 1250/1250 [00:08<00:00, 143.20it/s]

## GPU Usage for MLP-based Modeling

## Checking Model, Output and Log

In [13]:
!tree ./model/tag/

./model/tag/
├── mlp.model
└── mlp.model.vocab

1 directory, 2 files


In [14]:
!ls -lh ./model/tag/*

-rw-rw-r-- 1 ye ye 568K Jan 27 04:53 ./model/tag/mlp.model
-rw-rw-r-- 1 ye ye  110 Jan 27 04:52 ./model/tag/mlp.model.vocab


In [15]:
!wc ./model/tag/mlp.model.vocab

 17  34 110 ./model/tag/mlp.model.vocab


In [16]:
!cat ./model/tag/mlp.model.vocab

v	0
ppm	1
adv	2
part	3
conj	4
fw	5
[PAD]	6
tn	7
pron	8
num	9
adj	10
[UNK]	11
int	12
punc	13
abb	14
n	15
sb	16


## Tag Generation Results

In [17]:
!cat ./data/myPOS/tag/start_tags.txt

pron
n
adj
v
pron part
pron ppm
n v
n n
v part
n part
pron pron
pron ppm
n tn
adj v part
n n n


အထက်ပါ input ဖိုင်ထဲက start tag တွေကို အခြေခံပြီး MLP-based LM က generated လုပ်ပြီးထွက်လာတဲ့ output က အောက်ပါအတိုင်းပါ။  

In [18]:
!cat -n ./output/tag/mlp_gen_texts.txt

     1	pron conj v
     2	pron conj sb
     3	pron int fw
     4	pron [UNK] abb
     5	pron sb sb
     6	n v conj
     7	n n punc
     8	n abb v
     9	n abb abb
    10	n n pron
    11	adj v adv
    12	adj sb v
    13	adj adv [PAD]
    14	adj adj adv
    15	adj punc abb
    16	v num sb
    17	v tn punc
    18	v num n
    19	v tn abb
    20	v part punc
    21	pron part [UNK] v
    22	pron part fw int
    23	pron part num conj
    24	pron part conj punc
    25	pron part tn punc
    26	pron ppm sb conj
    27	pron ppm part fw
    28	pron ppm v adv
    29	pron ppm conj ppm
    30	pron ppm fw adj
    31	n v tn num
    32	n v conj v
    33	n v tn abb
    34	n v [UNK] num
    35	n v adv punc
    36	n n v [PAD]
    37	n n int [UNK]
    38	n n [PAD] v
    39	n n abb tn
    40	n n adv fw
    41	v part tn pron
    42	v part pron v
    43	v part part [PAD]
    44	v part pron fw
    45	v part tn [UNK]
    46	n part abb fw
    47	n part v part
    48	n part [PAD] pron
    49	n part punc adj
    50	n

tag generation ကို ဥပမာ အလုံး ၃၀ ထားကြည့်ပြီး run ကြည့်ရအောင်။  

In [20]:
!time python laphet.py --model_type mlp --generate --model ./model/tag/mlp.model \
--seq_len 30 --input ./data/myPOS/tag/start_tags.txt --no_of_generation 1

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: pron sb num adj tn [PAD] ppm adj 

Test data နဲ့ evaluation လုပ်ထားတဲ့ ရလဒ်ကို ကြည့်ကြည့်ရအောင်။  

In [21]:
!tail -n 2 ./log/tag/mlp.log

Average Perplexity on Test Data: 1.1039
Average Cross-Entropy on Test Data: 0.0988


## Bi-LSTM based LM Building, Tag Generation and Testing

bash shell script ကို အောက်ပါအတိုင်း updated လုပ်ခဲ့။  

# Run tasks for each model type in the specified order
#task mlp  
task bilstm  
#task transformer  
#task bert  
#task gpt  

In [22]:
!./train_test_tag.sh

Training Bilstm language model:
Epoch 1/10 (Training): 100%|████████████████| 1250/1250 [00:21<00:00, 58.23it/s]
Epoch 1, Training Loss: 0.1715
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 166.28it/s]
Epoch 1, Validation Loss: 0.0017
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0017
Epoch 2/10 (Training): 100%|████████████████| 1250/1250 [00:21<00:00, 59.38it/s]
Epoch 2, Training Loss: 0.0012
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 174.90it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0001
Epoch 3/10 (Training): 100%|████████████████| 1250/1250 [00:21<00:00, 58.66it/s]
Epoch 3, Training Loss: 0.0002
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 172.65it/s]
Epoch 3, Validation Loss: 0.0000
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0000
Epoch 4/10 (Training): 100%|████████████████| 1250/1250 [00:21<00:00

## Updated the Shell Script

In [23]:
!cat ./train_test_tag.sh

#!/bin/bash

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 10 --epochs 10 --batch_size 32 --lr 0.0001;

    echo "Text generation:";
    time python -u laphet.py --model_type $model_type --generate --model $model_file \
      --seq_len 5 --prompt "ရဲ" --no_of_generation 

## Training Again for Bi-LSTM

In [24]:
!./train_test_tag.sh

Training Bilstm language model:
Epoch 1/10 (Training): 100%|███████████████| 1250/1250 [00:07<00:00, 164.23it/s]
Epoch 1, Training Loss: 0.2847
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 600.32it/s]
Epoch 1, Validation Loss: 0.0010
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0010
Epoch 2/10 (Training): 100%|███████████████| 1250/1250 [00:07<00:00, 171.63it/s]
Epoch 2, Training Loss: 0.0013
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 564.93it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0001
Epoch 3/10 (Training): 100%|███████████████| 1250/1250 [00:07<00:00, 172.75it/s]
Epoch 3, Training Loss: 0.0004
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 553.96it/s]
Epoch 3, Validation Loss: 0.0000
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0000
Epoch 4/10 (Training): 100%|███████████████| 1250/1250 [00:07<00:00,

## Updated the Bash Shell Script

In [25]:
!cat ./train_test_tag.sh

#!/bin/bash

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001;

    echo "Text generation:";
    time python -u laphet.py --model_type $model_type --generate --model $model_file \
      --seq_len 50 --prompt "ရဲ" --no_of_generation

## Transformer based LM Training, Tag Generation and Testing

In [27]:
!./train_test_tag.sh

Training Transformer language model:
Epoch 1/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 209.48it/s]
Epoch 1, Training Loss: 0.0363
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 612.30it/s]
Epoch 1, Validation Loss: 0.0004
Best model saved at ./model/tag/transformer.model with validation loss: 0.0004
Epoch 2/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 222.14it/s]
Epoch 2, Training Loss: 0.0004
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 620.60it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/transformer.model with validation loss: 0.0001
Epoch 3/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 220.49it/s]
Epoch 3, Training Loss: 0.0002
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 622.85it/s]
Epoch 3, Validation Loss: 0.0001
Best model saved at ./model/tag/transformer.model with validation loss: 0.0001
Epoch 4/10 (Training): 100%|███████████████| 125

## Checking Model, Generated Tags and Log

In [28]:
!tree ./model/tag/

./model/tag/
├── bilstm.model
├── bilstm.model.vocab
├── mlp.model
├── mlp.model.vocab
├── transformer.model
└── transformer.model.vocab

1 directory, 6 files


In [29]:
!ls -lh ./model/tag/transformer.model*

-rw-rw-r-- 1 ye ye 8.2M Jan 27 05:34 ./model/tag/transformer.model
-rw-rw-r-- 1 ye ye  110 Jan 27 05:33 ./model/tag/transformer.model.vocab


In [30]:
!tree ./output/tag/

./output/tag/
├── bilstm_gen_texts.txt
├── mlp_gen_texts.txt
└── transformer_gen_texts.txt

1 directory, 3 files


In [32]:
!cat -n ./output/tag/transformer_gen_texts.txt

     1	pron part num
     2	pron ppm punc
     3	pron punc tn
     4	pron abb abb
     5	pron fw adj
     6	n [PAD] tn
     7	n adj conj
     8	n adj adv
     9	n sb part
    10	n sb v
    11	adj num n
    12	adj ppm adv
    13	adj v tn
    14	adj part num
    15	adj num part
    16	v n sb
    17	v fw sb
    18	v part fw
    19	v part [PAD]
    20	v conj adv
    21	pron part tn num
    22	pron part fw pron
    23	pron part tn adv
    24	pron part adv int
    25	pron part int pron
    26	pron ppm adv adj
    27	pron ppm v part
    28	pron ppm [UNK] tn
    29	pron ppm num [UNK]
    30	pron ppm fw v
    31	n v v pron
    32	n v adv v
    33	n v sb [UNK]
    34	n v adj tn
    35	n v num part
    36	n n num fw
    37	n n [UNK] [PAD]
    38	n n v int
    39	n n v int
    40	n n abb ppm
    41	v part n adj
    42	v part adj abb
    43	v part fw part
    44	v part punc sb
    45	v part tn n
    46	n part conj v
    47	n part v abb
    48	n part [UNK] tn
    49	n part ppm ppm
    50	n part tn [

In [34]:
!tail -n 2 ./log/tag/transformer.log

Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000


## Updated Bash Shell Script

#task mlp  
#task bilstm  
#task transformer  
task bert  
#task gpt  

## BERT based LM Building, Tag Generation and Testing

In [42]:
!./train_test_tag.sh

Training Bert language model:
Epoch 1/10 (Training): 100%|███████████████| 1250/1250 [00:06<00:00, 206.16it/s]
Epoch 1, Training Loss: 0.0274
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 638.96it/s]
Epoch 1, Validation Loss: 0.0004
Best model saved at ./model/tag/bert.model with validation loss: 0.0004
Epoch 2/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 223.56it/s]
Epoch 2, Training Loss: 0.0004
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 639.68it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/bert.model with validation loss: 0.0001
Epoch 3/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 228.74it/s]
Epoch 3, Training Loss: 0.0001
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 567.21it/s]
Epoch 3, Validation Loss: 0.0001
Best model saved at ./model/tag/bert.model with validation loss: 0.0001
Epoch 4/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 223.12i

## GPU Usage of BERT

## Checking Model, Generated Tags and Logs

In [43]:
!tree ./model/tag/

./model/tag/
├── bert.model
├── bert.model.vocab
├── bilstm.model
├── bilstm.model.vocab
├── mlp.model
├── mlp.model.vocab
├── transformer.model
└── transformer.model.vocab

1 directory, 8 files


In [45]:
!ls -lh ./model/tag/*.model

-rw-rw-r-- 1 ye ye 8.2M Jan 27 05:47 ./model/tag/bert.model
-rw-rw-r-- 1 ye ye  85M Jan 27 05:30 ./model/tag/bilstm.model
-rw-rw-r-- 1 ye ye 568K Jan 27 04:53 ./model/tag/mlp.model
-rw-rw-r-- 1 ye ye 8.2M Jan 27 05:34 ./model/tag/transformer.model


In [47]:
!cat -n ./output/tag/bert_gen_texts.txt

     1	pron adj fw
     2	pron [UNK] punc
     3	pron int adj
     4	pron conj ppm
     5	pron n int
     6	n abb conj
     7	n conj adv
     8	n num n
     9	n fw adj
    10	n int num
    11	adj n abb
    12	adj adj [PAD]
    13	adj pron adv
    14	adj fw int
    15	adj adj [UNK]
    16	v conj int
    17	v fw adv
    18	v punc part
    19	v [PAD] fw
    20	v n tn
    21	pron part abb abb
    22	pron part adj [PAD]
    23	pron part adj fw
    24	pron part num sb
    25	pron part num punc
    26	pron ppm fw int
    27	pron ppm adj abb
    28	pron ppm tn ppm
    29	pron ppm abb conj
    30	pron ppm pron [UNK]
    31	n v adj part
    32	n v v abb
    33	n v tn [PAD]
    34	n v int int
    35	n v tn num
    36	n n [PAD] n
    37	n n adj [PAD]
    38	n n part adj
    39	n n int part
    40	n n conj num
    41	v part sb fw
    42	v part adv part
    43	v part ppm num
    44	v part adj sb
    45	v part [PAD] fw
    46	n part n pron
    47	n part v ppm
    48	n part fw adj
    49	n part n sb
 

In [48]:
!cat ./log/tag/bert.log

Training Bert language model:
Epoch 1, Training Loss: 0.0274
Epoch 1, Validation Loss: 0.0004
Best model saved at ./model/tag/bert.model with validation loss: 0.0004
Epoch 2, Training Loss: 0.0004
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/bert.model with validation loss: 0.0001
Epoch 3, Training Loss: 0.0001
Epoch 3, Validation Loss: 0.0001
Best model saved at ./model/tag/bert.model with validation loss: 0.0001
Epoch 4, Training Loss: 0.0001
Epoch 4, Validation Loss: 0.0000
Best model saved at ./model/tag/bert.model with validation loss: 0.0000
Epoch 5, Training Loss: 0.0000
Epoch 5, Validation Loss: 0.0000
Best model saved at ./model/tag/bert.model with validation loss: 0.0000
Epoch 6, Training Loss: 0.0000
Epoch 6, Validation Loss: 0.0000
Best model saved at ./model/tag/bert.model with validation loss: 0.0000
Epoch 7, Training Loss: 0.0000
Epoch 7, Validation Loss: 0.0000
Best model saved at ./model/tag/bert.model with validation loss: 0.0000
Epoch 8, Training 

## Updated Bash Shell Script

#task mlp  
#task bilstm  
#task transformer  
#task bert  
task gpt  

## GPT based LM Training, Tag Generation and Testing

In [49]:
!./train_test_tag.sh

Training Gpt language model:
Epoch 1/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 214.73it/s]
Epoch 1, Training Loss: 0.0148
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 557.06it/s]
Epoch 1, Validation Loss: 0.0002
Best model saved at ./model/tag/gpt.model with validation loss: 0.0002
Epoch 2/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 231.47it/s]
Epoch 2, Training Loss: 0.0002
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 606.25it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/gpt.model with validation loss: 0.0001
Epoch 3/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 226.48it/s]
Epoch 3, Training Loss: 0.0001
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 595.10it/s]
Epoch 3, Validation Loss: 0.0000
Best model saved at ./model/tag/gpt.model with validation loss: 0.0000
Epoch 4/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 228.16it/s]

## GPU Usage of GPT LM Modeling

## Checking Model, Generated Tags, Log

In [50]:
!tree ./model/tag/

./model/tag/
├── bert.model
├── bert.model.vocab
├── bilstm.model
├── bilstm.model.vocab
├── gpt.model
├── gpt.model.vocab
├── mlp.model
├── mlp.model.vocab
├── transformer.model
└── transformer.model.vocab

1 directory, 10 files


In [51]:
!ls -lh ./model/tag/gpt*

-rw-rw-r-- 1 ye ye 8.2M Jan 27 05:51 ./model/tag/gpt.model
-rw-rw-r-- 1 ye ye  110 Jan 27 05:50 ./model/tag/gpt.model.vocab


In [52]:
!cat -n ./output/tag/gpt_gen_texts.txt

     1	pron adj [UNK]
     2	pron abb tn
     3	pron punc [PAD]
     4	pron [UNK] part
     5	pron fw int
     6	n adj punc
     7	n punc pron
     8	n tn part
     9	n adj punc
    10	n pron n
    11	adj part adv
    12	adj num adv
    13	adj int sb
    14	adj punc conj
    15	adj adj abb
    16	v int n
    17	v tn [PAD]
    18	v ppm adj
    19	v conj ppm
    20	v pron conj
    21	pron part adj adv
    22	pron part abb pron
    23	pron part sb [UNK]
    24	pron part sb [UNK]
    25	pron part tn sb
    26	pron ppm part sb
    27	pron ppm n ppm
    28	pron ppm conj fw
    29	pron ppm v v
    30	pron ppm [UNK] conj
    31	n v part tn
    32	n v n abb
    33	n v [PAD] abb
    34	n v [PAD] int
    35	n v fw n
    36	n n sb tn
    37	n n ppm part
    38	n n conj n
    39	n n part abb
    40	n n adv fw
    41	v part pron sb
    42	v part [UNK] sb
    43	v part punc part
    44	v part part tn
    45	v part tn conj
    46	n part [UNK] v
    47	n part conj ppm
    48	n part num num
    49	n par

In [53]:
!tail -n 2 ./log/tag/*.log

==> ./log/tag/bert.log <==
Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000

==> ./log/tag/bilstm.log <==
Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000

==> ./log/tag/gpt.log <==
Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000

==> ./log/tag/mlp.log <==
Average Perplexity on Test Data: 1.1039
Average Cross-Entropy on Test Data: 0.0988

==> ./log/tag/transformer.log <==
Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000
